# Getting started with Mapper

In this notebook we explore a few of the core features included in ``giotto-tda``'s implementation of the [Mapper algorithm](https://research.math.osu.edu/tgda/mapperPBG.pdf). 

If you are looking at a static version of this notebook and would like to run its contents, head over to [GitHub](https://github.com/giotto-ai/giotto-tda/blob/master/examples/mapper_quickstart.ipynb) and download the source.

## Useful references

* [An introduction to Topological Data Analysis: fundamental and practical aspects for data scientists](https://arxiv.org/abs/1710.04019)
* [An Introduction to Topological Data Analysis for Physicists: From LGM to FRBs](https://arxiv.org/abs/1904.11044)

**License: AGPLv3**

## Import libraries

In [ ]:
# Data wrangling
import numpy as np
import pandas as pd  # Not a requirement of giotto-tda, but is compatible with the gtda.mapper module

# Data viz
from gtda.plotting import plot_point_cloud

# TDA magic
from gtda.mapper import (
    CubicalCover,
    make_mapper_pipeline,
    Projection,
    plot_static_mapper_graph,
    plot_interactive_mapper_graph,
    MapperInteractivePlotter
)

# ML tools
from sklearn import datasets
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

## Generate and visualise data
As a simple example, let's generate a two-dimensional point cloud of two concentric circles. The goal will be to examine how Mapper can be used to generate a topological graph that captures the salient features of the data.

In [ ]:
data, _ = datasets.make_circles(n_samples=5000, noise=0.05, factor=0.3, random_state=42)

plot_point_cloud(data)

## Configure the Mapper pipeline
Given a dataset ${\cal D}$ of points $x \in \mathbb{R}^n$, the basic steps behind Mapper are as follows:

1. Map ${\cal D}$ to a lower-dimensional space using a **filter function** $f: \mathbb{R}^n \to \mathbb{R}^m$. Common choices for the filter function include projection onto one or more axes via PCA or density-based methods. In ``giotto-tda``, you can import a variety of filter functions as follows:

```python
from gtda.mapper.filter import FilterFunctionName
```

2. Construct a cover of the filter values ${\cal U} = (U_i)_{i\in I}$, typically in the form of a set of overlapping intervals which have constant length. As with the filter, a choice of cover can be imported as follows:

```python
from gtda.mapper.cover import CoverName
```

3. For each interval $U_i \in {\cal U}$ cluster the points in the preimage $f^{-1}(U_i)$ into sets $C_{i,1}, \ldots , C_{i,k_i}$. The choice of clustering algorithm can be any of ``scikit-learn``'s [clustering methods](https://scikit-learn.org/stable/modules/clustering.html) or an implementation of agglomerative clustering in ``giotto-tda``:

```python
# scikit-learn method
from sklearn.cluster import ClusteringAlgorithm
# giotto-tda method
from gtda.mapper.cluster import FirstSimpleGap
```

4. Construct the topological graph whose vertices are the cluster sets $(C_{i,j})_{i\in I, j \in \{1,\ldots,k_i\}}$ and an edge exists between two nodes if they share points in common: $C_{i,j} \cap C_{k,l} \neq \emptyset$. This step is handled automatically by ``giotto-tda``.

These four steps are implemented in the ``MapperPipeline``  object that mimics the ``Pipeline`` class from ``scikit-learn``. We provide a convenience function ``make_mapper_pipeline`` that allows you to pass the choice of filter function, cover, and clustering algorithm as arguments. For example, to project our data onto the $x$- and $y$-axes, we could setup the pipeline as follows:

In [ ]:
# Define filter function – can be any scikit-learn transformer
filter_func = Projection(columns=[0, 1])
# Define cover
cover = CubicalCover(n_intervals=10, overlap_frac=0.3)
# Choose clustering algorithm – default is DBSCAN
clusterer = DBSCAN()

# Configure parallelism of clustering step
n_jobs = 1

# Initialise pipeline
pipe = make_mapper_pipeline(
    filter_func=filter_func,
    cover=cover,
    clusterer=clusterer,
    verbose=False,
    n_jobs=n_jobs,
)

## Visualise the Mapper graph

With the Mapper pipeline at hand, it is now a simple matter to visualise it. To warm up, let's examine the graph in two-dimensions using the default arguments of ``giotto-tda``'s plotting function:

In [ ]:
fig = plot_static_mapper_graph(pipe, data)
fig.show(config={'scrollZoom': True})

By default, nodes are coloured according to the average row index of the data points they represent.

From the figure we can see that we have captured the salient topological features of our underlying data, namely two holes!

### Configure the colouring of the Mapper graph

In this example, it is more instructive to colour by the average values of the $x$- and $y$-coordinates. This can be achieved by passing the input data again as the keyword argument ``color_data``. In general, any ``numpy`` array or ``pandas`` dataframe explicitly passed as ``color_data`` will be used to calculate one colouring per column present. A dropdown menu is automatically created if ``color_data`` has more than one column, to easily switch between column-based colourings.

At the same time, let's configure the choice of colorscale:

In [ ]:
plotly_params = {"node_trace": {"marker_colorscale": "Blues"}}
fig = plot_static_mapper_graph(
    pipe, data, color_data=data, plotly_params=plotly_params
)
fig.show(config={'scrollZoom': True})

Even finer control over the colouring can be achieved by making use of the additional keyword arguments ``color_features`` and ``node_color_statistic``. The former can be a ``scikit-learn`` transformer or a list of indices or column names to select from the data. For example, coloring by a PCA component can be neatly implemented as follows:

In [ ]:
# Initialise estimator to color graph by
pca = PCA(n_components=1)

fig = plot_static_mapper_graph(
    pipe, data, color_data=data, color_features=pca
)
fig.show(config={'scrollZoom': True})

``node_color_statistic`` refers to the function used to extract single colour values for each node, starting from the values of ``color_features`` at each data point. The default, as we have seen, is ``np.mean``. But any other callable is acceptable which sends *column vectors* to scalars:

In [ ]:
fig = plot_static_mapper_graph(
    pipe, data, color_data=data, color_features=pca, node_color_statistic=lambda x: np.mean(x) / 2
)
fig.show(config={'scrollZoom': True})

If you prefer to just input custom node colours directly, you can do so by passing a ``numpy`` array or ``pandas`` dataframe of the correct length as ``node_color_statistic``. For example (see "Run the Mapper pipeline" below), the Mapper nodes for this particular Mapper pipeline would be as follows:

In [ ]:
graph = pipe.fit_transform(data)
node_elements = graph.vs["node_elements"]
print(f"There are {len(node_elements)} nodes.\nThe first node consists of row indices {node_elements[0]}.")

Let us try this:

In [ ]:
fig = plot_static_mapper_graph(
    pipe, data, node_color_statistic=np.arange(len(node_elements))
)
fig.show(config={'scrollZoom': True})

### Pass a pandas DataFrame as input

It is also possible to feed ``plot_static_mapper_graph`` a pandas DataFrame:

In [ ]:
df = pd.DataFrame(data, columns=["x", "y"])
df.head()

Before plotting we need to update the Mapper pipeline to know about the projection onto the column names. This can be achieved using the ``set_params`` method as follows:

In [ ]:
pipe.set_params(filter_func=Projection(columns=["x", "y"]));

In [ ]:
fig = plot_static_mapper_graph(pipe, df, color_data=df)
fig.show(config={'scrollZoom': True})

### Example: colour by proportions of categorical variables

Often one has a dataset of observations, each belonging to a category (e.g. a country or region name). It can be very useful to visualize the distributions of each category in the nodes of the Mapper graph. As a trivial example, let us add a categorical column to our dataframe, with value equal to ``"A"`` for the outer circle, and ``"B"`` for the inner one: 

In [ ]:
df["Circle"] = df["x"] ** 2 + df["y"] ** 2 < 0.25
df["Circle"] = df["Circle"].replace([False, True], ["A", "B"])

To visualize the proportions of data points in each Mapper node belonging to either circle, we can create a dataframe of one-hot encodings of the categorical variable ``"Circle"``, and pass it to ``plot_static_mapper_graph`` as ``color_data``:

In [ ]:
color_data = pd.get_dummies(df["Circle"], prefix="Circle")

fig = plot_static_mapper_graph(pipe, df[["x", "y"]], color_data=color_data)
fig.show(config={'scrollZoom': True})

The dropdown menu allows us to quickly switch colourings according to each category, without needing to recompute the underlying graph.

### Change the layout algorithm

By default, ``plot_static_mapper_graph`` uses the Kamada–Kawai algorithm for the layout; however any of the layout algorithms defined in python-igraph are supported (see [here](https://igraph.org/python/doc/igraph.Graph-class.html) for a list of possible layouts). For example, we can switch to the Fruchterman–Reingold layout as follows:

In [ ]:
# Reset back to numpy projection
pipe.set_params(filter_func=Projection(columns=[0, 1]));

In [ ]:
fig = plot_static_mapper_graph(
    pipe, data, layout="fruchterman_reingold", color_data=data
)
fig.show(config={'scrollZoom': True})

### Change the layout dimension

It is also possible to visualise the Mapper graph in 3 dimensions by configuring the ``layout_dim`` argument:

In [ ]:
fig = plot_static_mapper_graph(pipe, data, layout_dim=3, color_data=data)
fig.show(config={'scrollZoom': True})

### Change the node size scale

In general, node sizes are proportional to the number of dataset elements contained in the nodes. Sometimes, however, the default scale leads to graphs which are difficult to decipher, due to e.g. excessively small nodes. The ``node_scale`` parameter can be used to configure this scale.

In [ ]:
node_scale = 30
fig = plot_static_mapper_graph(pipe, data, layout_dim=3, node_scale=node_scale)
fig.show(config={'scrollZoom': True})

## Run the Mapper pipeline

Behind the scenes of ``plot_static_mapper_graph`` is a ``MapperPipeline`` object ``pipe`` that can be used like a typical ``scikit-learn`` estimator. For example, to extract the underlying graph data structure we can do the following:

In [ ]:
graph = pipe.fit_transform(data)

The resulting graph is a [python-igraph](https://igraph.org/python/) object which stores node metadata in the form of attributes. We can access this data as follows:

In [ ]:
graph.vs.attributes()

Here ``'pullback_set_label'`` and ``'partial_cluster_label'`` refer to the interval and cluster sets described above. ``'node_elements'`` refers to the indices of our original data that belong to each node. For example, to find which points belong to the first node of the graph we can access the desired data as follows:

In [ ]:
node_id = 0
node_elements = graph.vs["node_elements"]

print(f"""
Node ID: {node_id}
Node elements: {node_elements[node_id]}
Data points: {data[node_elements[node_id]]}
""")

## Creating custom filter functions

In some cases, the list of filter functions provided in ``gtda.mapper.filter.py`` or ``scikit-learn`` may not be sufficient for the task at hand. In such cases, one can pass any callable to the pipeline that acts **row-wise** on the input data. For example, we can project by taking the sum of the $(x,y)$ coordinates as follows:

In [ ]:
filter_func = np.sum

pipe = make_mapper_pipeline(
    filter_func=filter_func,
    cover=cover,
    clusterer=clusterer,
    verbose=True,
    n_jobs=n_jobs,
)

In [ ]:
fig = plot_static_mapper_graph(pipe, data)
fig.show(config={'scrollZoom': True})

## Visualise the Mapper graph interactively (Live Jupyter session needed)

In general, building useful Mapper graphs requires some iteration through the various parameters in the cover and clustering algorithm. To simplify that process, ``giotto-tda`` provides an interactive figure that can be configured in real time by tweaking the pipeline hyperparameters. You can produce it in two ways, namely:

  1. by using the ``plot_interactive_mapper_graph`` function in a similar same way as ``plot_static_mapper_graph``;

In [ ]:
pipe = make_mapper_pipeline()

# Generate interactive widget
plot_interactive_mapper_graph(pipe, data, color_data=data)

  2. (**recommended**, new in ``giotto-tda`` 0.5.0) in an object-oriented way, by instantiating a ``MapperInteractivePlotter`` onject and then calling its ``plot`` method to create the widget.

In [ ]:
# Create the plotter object
MIP = MapperInteractivePlotter(pipe, data)

# Generate interactive widget
MIP.plot(color_data=data)

The advantage of using the class API with ``MapperInteractivePlotter`` is that, once you are done tweaking the hyperparameters, you can inspect the latest state of the objects (graph, colours, pipeline, inner figure) which got changed during the interactive session.

In [ ]:
print("Attributes created by `.plot` and updated during the interactive session:\n",
      [attr for attr in dir(MIP) if attr.endswith("_") and attr[0] != "_"])

In the widgets, if invalid parameters are selected, the _Show logs_ checkbox can be used to see what went wrong.

To see the interactive outputs above, please **download** the notebook from [GitHub](https://github.com/giotto-ai/giotto-tda/blob/master/examples/mapper_quickstart.ipynb) and execute it locally.